<center>
<img src="https://habrastorage.org/web/677/8e1/337/6778e1337c3d4b159d7e99df94227cb2.jpg"/>
## Специализация "Машинное обучение и анализ данных"
</center>

# <center> Идентификация пользователей по посещенным веб-страницам</center>
<center> <img align=center src='http://i.istockimg.com/file_thumbview_approve/21546327/5/stock-illustration-21546327-identification-de-l-utilisateur.jpg'></center>
<center>Автор материала: Пугачева Алиса</center>

Идея такая: пользователи Интернета по-разному переходят по ссылкам, и это может помогать их идентифицировать. В качестве входных данных мы имеем несколько файлов, соответствующих пользователям, каждый из которых содержит таблицу вида:

## Описание исходных и обработанных данных и признаков

In [1]:
import pandas as pd
pd.read_csv('capstone_user_identification/3users/user0001.csv').head(2)

,timestamp,site
0,2013-11-15 09:28:17,vk.com
1,2013-11-15 09:33:04,oracle.com


В [соревновании](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) на Kaggle входные данные предобработанны таким образом, что представляют из себя таблицы где строка - последовательность веб-страниц, метки времени захода на соотвествующий сайт и меткой пользователя:

In [2]:
import os
PATH_TO_DATA = ('capstone_user_identification')

train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id', parse_dates=['time1'])
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id', parse_dates=['time1'])

train_df.head(2)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0


Посмотрим, как распределены по времени сессии в обучающей и тестовой выборке:

In [3]:
train_df.time1.min(), train_df.time1.max(), test_df.time1.min(), test_df.time1.max()

(Timestamp('2013-01-12 08:05:57'),
 Timestamp('2014-04-30 23:39:53'),
 Timestamp('2014-05-01 17:14:03'),
 Timestamp('2014-12-05 23:26:53'))

Видим, что временные отрезки(начала сессий) не пересекаются.
В связи с этим, сортируем сессии в обучающей выборке по времени (по полю time1)

In [4]:
train_df = train_df.sort_values(by='time1')
train_df.head(2)

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 21669 to 204762
Data columns (total 21 columns):
site1     253561 non-null int64
time1     253561 non-null datetime64[ns]
site2     250098 non-null float64
time2     250098 non-null object
site3     246919 non-null float64
time3     246919 non-null object
site4     244321 non-null float64
time4     244321 non-null object
site5     241829 non-null float64
time5     241829 non-null object
site6     239495 non-null float64
time6     239495 non-null object
site7     237297 non-null float64
time7     237297 non-null object
site8     235224 non-null float64
time8     235224 non-null object
site9     233084 non-null float64
time9     233084 non-null object
site10    231052 non-null float64
time10    231052 non-null object
target    253561 non-null int64
dtypes: datetime64[ns](1), float64(9), int64(2), object(9)
memory usage: 42.6+ MB


Сессии ограничены количеством сайтов (10) или длительностью сессии (30 мин)

Судя по входным данным, некоторые строки имеют пропуски, когда длительность сессии больше получаса.

Кроме train- и test- таблиц, у нас есть словарь всех сайтов site_dict:

In [6]:
import pickle
with open('capstone_user_identification/site_dic.pkl', 'rb') as sd:
    site_dict = pickle.load(sd)
print(pd.Series(site_dict)[:10])
'Длина словаря сайтов: {}'.format(len(site_dict))

0.0-02.net                     24314
0.56.web1.im.weibo.com         43357
0.academia-assets.com          18340
0.academia-photos.com          32767
0.api.urban-rivals.com         35936
0.client-channel.google.com    47984
0.cyberprofs.com                1542
0.docs.google.com                775
0.drive.google.com               678
0.envato-static.com            30517
dtype: int64


'Длина словаря сайтов: 48371'

In [7]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

data = [go.Bar(y=list(train_df.target.value_counts()), x=train_df.target.value_counts().index)]
iplot(data)

Видим, что в обучающей выборке классы очень несбалансированы: один класс больше другого примерно в сто раз.

Объеденим обучающую и тестовую выборки без меток класса, чтобы производить преобразования и добавление новых признаков:

In [8]:
full_df = pd.concat([train_df.drop('target', axis=1), test_df])
full_df.shape

(336358, 20)

Выделим колонки сайтов и времени и поменяем тип колонок сайтов на int, заодно заменим пропуски сайтов на 0, так как нумерация сайтов в словаре site_dict начинается с единицы:

In [9]:
time_cols = ['time%s' % i for i in range(1, 11)]
site_cols = ['site%s' % i for i in range(1, 11)]
train_df[site_cols] = train_df[site_cols].fillna(0).astype(int)
full_df[site_cols] = full_df[site_cols].fillna(0).astype(int)
idx = train_df.shape[0]
full_df[site_cols].head(3)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946


In [10]:
test_df[site_cols] = test_df[site_cols].fillna(0).astype(int)
test_df[site_cols].head(3)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,29,35,22,321,23,2211,6730,21,44582,15336
2,782,782,782,782,782,782,782,782,782,782
3,55,55,55,55,55,55,55,55,1445,1445


## Начнем создавать новые признаки

In [11]:
import numpy as np
additional_data_df = pd.DataFrame(index = full_df.index)

### Количество уникальных сайтов в сессиях

Посмотрим на распределение количества уникальных сайтов в сессиях, и как оно различается в зависимости от пользователя:

In [12]:
train_df[site_cols].head(5).apply(lambda x: len(np.unique(x[x!=0])), axis=1)

session_id
21669     2
54843     2
77292     6
114021    5
146670    6
dtype: int64

In [13]:
train_df[site_cols].head(5)

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [14]:
unique_count_sites_df = pd.DataFrame({'nunique_site_count' : train_df[site_cols].apply(lambda x: len(np.unique(x[x!=0])), axis=1), 'target' : train_df.target})
unique_count_sites_df.head()

,nunique_site_count,target
session_id,,
21669,2,0
54843,2,0
77292,6,0
114021,5,0
146670,6,0


In [15]:
data4 = [ go.Bar(
    x=unique_count_sites_df[unique_count_sites_df.target == 0]['nunique_site_count'].value_counts().index,
    y=unique_count_sites_df[unique_count_sites_df.target == 0]['nunique_site_count'].value_counts(),
    name='target 0'
), go.Bar(
    x=unique_count_sites_df[unique_count_sites_df.target == 1]['nunique_site_count'].value_counts().index,
    y=unique_count_sites_df[unique_count_sites_df.target == 1]['nunique_site_count'].value_counts(),
    name='target 1'
)]
layout = {'title': 'Distributions of amount of unique sites in sessions'}
fig = go.Figure(data=data4, layout=layout)
iplot(fig)

Добавим столбец 'nunique' в датафрейм additional_data_df:

In [16]:
#del unique_count_sites_df
additional_data_df['nunique'] = full_df[site_cols].apply(lambda x: len(np.unique(x[x!=0])), axis=1)

### Индикаторы популярных сайтов в сессии

#### Найдем наиболее популярные сайты в обучающей выборке среди всех пользователей

In [17]:
top_sites = pd.Series(train_df[site_cols].values.flatten()
                     ).value_counts().sort_values(ascending=False).head(10)
top_sites

21     123776
0      122730
23      87619
782     77055
22      58258
29      54094
167     46405
780     43841
778     38194
52      36085
dtype: int64

In [18]:
# Alice's preferences
top_sites_alice = pd.Series(train_df[train_df.target==1][site_cols].values.flatten()
                           ).value_counts().sort_values(ascending=False).head(10)
top_sites_alice

77     1382
80     1354
76     1307
29      897
21      857
81      609
879     522
22      522
75      451
82      447
dtype: int64

Заметим, что сайты 21, 22 и 29 находятся в топе популярных и у Alice, и у остальных пользователей.

Посмотрим, как часто встречаются сайты, популярные у Alice, среди всех сессий выборки:

In [19]:
alices_pop_sites=[]
for s in top_sites_alice.index:
    alices_pop_sites.append(len(train_df[train_df.target==0][site_cols].values.flatten()[train_df[train_df.target==0][site_cols].values.flatten()==s]))

In [20]:
alices_top_names = []
for s in top_sites_alice.index:
    alices_top_names.append(list(site_dict.keys())[list(site_dict.values()).index(s)])

In [21]:
alices_top_names

['i1.ytimg.com',
 's.youtube.com',
 'www.youtube.com',
 'www.facebook.com',
 'www.google.fr',
 'r4---sn-gxo5uxg-jqbe.googlevideo.com',
 'r1---sn-gxo5uxg-jqbe.googlevideo.com',
 'apis.google.com',
 's.ytimg.com',
 'r2---sn-gxo5uxg-jqbe.googlevideo.com']

In [22]:
data5 = [
    go.Bar(y=top_sites_alice, x=alices_top_names, name='among Alice'),
    go.Bar(y=alices_pop_sites, x=alices_top_names, name='among all another')
]
fig = go.Figure(data=data5, layout={'title': 'Distribution of alices most popular sites'})
iplot(fig)

Видим, что сайты, популярные у Alice, пользуются популярностью также и у остальных пользователей. Тем не менее, они будут нам полезны, добавим в датафрейм  additional_data_df индикатор захода на эти сайты:

In [23]:
additional_data_df['alices_top_site'] = full_df[site_cols].apply(lambda x: 1 if x.any() in list(top_sites_alice.index) else 0, axis=1)

Из-за того, что списки пересекаются, начиная уже с четвертого элемента, возможно, более показательно будет добавить индикатор захода в топ-3 популярных сайтов Alice.

In [24]:
additional_data_df['alices_top3_site'] = full_df[site_cols].apply(lambda x: 1 if x.any() in list(top_sites_alice.index)[:3] else 0, axis=1)

In [25]:
additional_data_df

,nunique,alices_top_site,alices_top3_site
session_id,,,
21669,2,0,0
54843,2,0,0
77292,6,0,0
114021,5,0,0
146670,6,0,0
242171,5,0,0
57157,5,0,0
240201,6,0,0
210686,5,0,0


Кроме того, добавим индикатор захода на наиболее популярные сайты в выборке:

In [26]:
additional_data_df['top_site'] = full_df[site_cols].apply(lambda x: 1 if x.any() in list(top_sites.index) else 0, axis=1)

### День недели, час начала сессии, часть дня, индикатор выходного дня

In [27]:
additional_data_df['start_hour'] = full_df['time1'].apply(lambda x: x.hour)

In [28]:
additional_data_df['day_of_week'] = full_df['time1'].apply(lambda x: x.weekday())

In [29]:
import seaborn as sns
import matplotlib.pyplot as plt
_, axes = plt.subplots(1, 2, figsize=(16,6))

sns.countplot(x=additional_data_df[:idx][train_df.target==0].day_of_week, hue=train_df.target, ax=axes[0]);
sns.countplot(x=additional_data_df[:idx][train_df.target==1].day_of_week, hue=train_df.target, ax=axes[1]);
_.suptitle('Distribution sessions by day of week'); # can also get the figure from plt.gcf()

In [30]:
import seaborn as sns
_, axes = plt.subplots(1, 2, figsize=(16,6))

sns.countplot(x=additional_data_df[:idx][train_df.target==0].start_hour, hue=train_df.target, ax=axes[0]);
sns.countplot(x=additional_data_df[:idx][train_df.target==1].start_hour, hue=train_df.target, ax=axes[1]);
_.suptitle('Distribution sessions by day of week'); # can also get the figure from plt.gcf()

Видим, что у Alice довольно отличное от общей выборки время начала сессии. Укажем отдельным столбцом часть дня, соответсвующую start_hour:

In [31]:
def day_part(hour):
    if hour<9:
        return 'early_morning'
    elif hour<12:
            return 'morning'
    elif hour<16:
            return 'day'
    elif hour<19:
            return 'early evening'
    else:
        return 'evening'

In [32]:
additional_data_df['daypart'] = additional_data_df['start_hour'].apply(lambda x: day_part(x))

In [33]:
additional_data_df['weekend'] = additional_data_df['day_of_week'].apply(lambda x: 1 if x>4 else 0)

### Признак захода на vk.com
Это мой "инсайт", который я отрыла, рассматривая файлы пользователей: некоторые пользователи вовсе не заходят на этот сайт, а Alice - довольно много

In [34]:
site_vk = site_dict['vk.com']

In [35]:
len(train_df[train_df.target==0][site_cols].values.flatten()[train_df[train_df.target==0][site_cols].values.flatten()==site_vk])

1035

In [36]:
len(train_df[train_df.target==1][site_cols].values.flatten()[train_df[train_df.target==1][site_cols].values.flatten()==site_vk])

286

In [37]:
additional_data_df['vk.com'] = full_df[site_cols].apply(lambda x: 1 if site_vk in x else 0, axis=1)

### Трансформация временных меток в интервалы времени

In [38]:
stamps = full_df[time_cols].apply(pd.to_datetime)
for t in stamps[:-1]:
    stamps[t] = np.log1p((stamps.shift(-1, axis=1)[t] - stamps[t])/np.timedelta64(1,'s'))
stamps.drop('time10', axis=1, inplace=True)

In [39]:
stamps.head()

,time1,time2,time3,time4,time5,time6,time7,time8,time9
session_id,,,,,,,,,
21669,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54843,0.000000,7.487174,1.098612,NaN,NaN,NaN,NaN,NaN,NaN
77292,0.693147,0.693147,0.000000,0.693147,0.0,0.000000,0.000000,0.693147,0.000000
114021,0.000000,0.693147,0.000000,0.000000,0.0,0.693147,0.000000,0.000000,0.693147
146670,0.000000,0.000000,0.693147,0.000000,0.0,0.000000,0.693147,0.000000,0.000000


Добавим в additional_data_df столбец timespan - разница между первым и последним сайтом в сессии.

In [40]:
additional_data_df['timespan'] = stamps.sum(axis=1)

## Преобразуем категориальные признаки

In [41]:
add_data_df = pd.get_dummies(additional_data_df, columns=['start_hour', 'day_of_week', 'daypart'])

In [42]:
add_data_df.head()

,nunique,alices_top_site,alices_top3_site,top_site,weekend,vk.com,timespan,start_hour_7,start_hour_8,start_hour_9,...,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,day_of_week_6,daypart_day,daypart_early evening,daypart_early_morning,daypart_evening,daypart_morning
session_id,,,,,,,,,,,,,,,,,,,,,
21669,2,0,0,0,1,0,0.000000,0,1,0,...,0,0,0,1,0,0,0,1,0,0
54843,2,0,0,0,1,0,8.585786,0,1,0,...,0,0,0,1,0,0,0,1,0,0
77292,6,0,0,0,1,0,2.772589,0,1,0,...,0,0,0,1,0,0,0,1,0,0
114021,5,0,0,0,1,0,2.079442,0,1,0,...,0,0,0,1,0,0,0,1,0,0
146670,6,0,0,0,1,0,1.386294,0,1,0,...,0,0,0,1,0,0,0,1,0,0


## Векторизуем данные сайтов

In [43]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1, 3), max_features=50000)
full_df[site_cols][:idx].to_csv('train_sessions_text.txt', sep=' ', index=None, header=None)
full_df[site_cols][idx:].to_csv('test_sessions_text.txt', sep=' ', index=None, header=None)
with open('train_sessions_text.txt') as inp_train_file:
    X_train_sites = cv.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test_sites = cv.transform(inp_test_file)
X_train_sites.shape, X_test_sites.shape

((253561, 50000), (82797, 50000))

In [44]:
vocabulary_list = list(cv.vocabulary_.values())

## Создадим разбиение с помощью TimeSeriesSplit

In [45]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
time_split = TimeSeriesSplit(n_splits=10)

## Обучение модели

### Объединяем все датафреймы в один, и разбиваем обратно на обучающую и тестовую выбобрки

In [46]:
from scipy.sparse import hstack, vstack

In [47]:
train_sparse = hstack([X_train_sites, add_data_df[:idx].values], 'csr')
test_sparse = hstack([X_test_sites, add_data_df[idx:].values], 'csr')

### Создаем несколько моделей

#### Обучаем модель RandomForestRegressor, и добавляем его предсказание в качестве нового столбца выборки. Таким образом реализуем ансамблирование моделей

In [48]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.ensemble import RandomForestRegressor

In [49]:
lr = LogisticRegression(C=1, solver='lbfgs', random_state=17)

In [50]:
params={
    'C' : np.logspace(-2, 1, 10),
   # 'penalty' : ['l2', 'l1'],
   # 'class_weight' : [None, 'balanced'],
    #'tol' : np.logspace(-4, 1, 10)
}
grid_search = GridSearchCV(estimator=lr, param_grid=params, scoring='roc_auc', cv=time_split, verbose=1, n_jobs=1)

In [51]:
grid_search.fit(train_sparse, train_df.target)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  4.7min finished


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=10),
       error_score='raise',
       estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=17, solver='lbfgs', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': array([ 0.01   ,  0.02154,  0.04642,  0.1    ,  0.21544,  0.46416,
        1.     ,  2.15443,  4.64159, 10.     ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [52]:
grid_search.best_score_, grid_search.best_params_

(0.9151109444682917, {'C': 0.21544346900318834})

In [53]:
test_pred3 = grid_search.predict_proba(test_sparse)[:, 1]

## Оценим важность признаков
Построим график для наиболее весомых признаков

In [57]:
importance_features_pos = np.argsort(grid_search.best_estimator_.coef_.ravel())[-20:]

importance_features_neg = np.argsort(grid_search.best_estimator_.coef_.ravel())[:20]

In [59]:
def feature_names(importance_features):
    xticks = []
    for f in importance_features:
        if f>=50000:
            xticks.append(add_data_df.columns[f-50000])
        else:
            sites = list(cv.vocabulary_.keys())[vocabulary_list.index(f)]
            harmon = []
            for s in sites.split():
                harmon.append(list(site_dict.keys())[list(site_dict.values()).index(int(s))]) 
            xticks.append(','.join(harmon))
    return(xticks)

In [ ]:
xticks

In [72]:
data6 = [
    go.Bar(y=grid_search.best_estimator_.coef_.ravel()[importance_features_neg], x=feature_names(importance_features_neg), name='positive features'),
    go.Bar(y=grid_search.best_estimator_.coef_.ravel()[importance_features_pos], x=feature_names(importance_features_pos), name='negative features')
]

fig = go.Figure(data=data6, layout={'title': 'Coefficients of most important featutes'})
iplot(fig)

Видим, что среди наиболее важных признаков преимущественно сайты, на которые, например, заходила только Alice, или, наоборот, те, на которые она не заходила никогда. Из мною сгенерированных признаков сюда попали только время (start_hour_12) и часть дня (daypart_early_morning)

### Попробуем обучить еще одну модель - RandomForestRegressor, и усреднить получившиеся результаты

In [54]:
rf = RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=17)
rf.fit(train_sparse, train_df.target)
rf_predict = rf.predict(test_sparse)

In [91]:
result_answer = np.mean([rf_predict, test_pred3], axis=0)

In [92]:
result_answer.shape

(82797,)

In [55]:
# A helper function for writing predictions to a file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [93]:
write_to_submission_file(result_answer, 'kaggle_answer_file.txt')

## Заключение. Итоги

В ходе проекта было реализованы следующие шаги:
- генерация новых признаков, таких как время начала сессии, количество уникальных сайтов в сессии, день недели, часть дня, время сессии
- предобработка данных включала в себя сортировку данных по времени, преобразование данных сайтов в разреженную матрицу, логарифмирование длительности сессии, преобразование порядковых признаков в категориальные.
- обучение модели логистической регрессии, обучение модели регрессии случайного леса, усреднение результатов.